In [ ]:
#pip install "camelot-py[cv]"
#pip install PyPDF2

In [125]:
import pandas as pd
import numpy as np
import plotly.express as px
import camelot
import PyPDF2
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [90]:
"""
--------------------------------------------------------
CON CAMELOT PASO PDF A DATAFRAME 
--------------------------------------------------------
RUTAS 
--------------------------------------------------------
"pdf\Producción por Grupo y Tratami 24.pdf"  ---> NO LEE TABLA FORMAS DE PAGO
"pdf\Producción por Grupo y Tratami 23.pdf"
--------------------------------------------------------
'lattice' ..... for tables with clear lines. ---> NO LO LEE CORRECTAMENTE
'stream'  ..... for tables with whitespace separation.
--------------------------------------------------------
"""

pdf_path_23 = "pdf\Producción por Grupo y Tratami 23.pdf"
pdf_path_24 = "pdf\Producción por Grupo y Tratami 24.pdf"

def convertir_pdf_a_dataframe(ruta, pages="all", flavor="stream"):

    tablas = camelot.read_pdf(ruta, pages=pages, flavor=flavor)

    if not tablas:
        print(f"No tables were found in {ruta}")  
    else:
        dataframes = []
        for tabla in tablas:
            dataframes.append(tabla.df)

    return dataframes

dataframes_23 = convertir_pdf_a_dataframe(pdf_path_23)
dataframes_24 = convertir_pdf_a_dataframe(pdf_path_24)

In [91]:
def formatear_tabla(dataframe, pagina):
    tabla = dataframe

    #DONDE ELEMENTO VACIO
    tabla[0] = tabla[0].replace(r'^\s*$', np.nan, regex=True)
    #RELLENAR CON NO Nan PREVIO
    tabla[0] = tabla[0].ffill()

    #ELIMINAR COLUMNA CODIGO
    tabla.drop(1, axis=1, inplace=True)

    if pagina == 1:
        #ELIMINAR TITULO
        tabla.drop(0, inplace=True)

        #ELIMINAR FILAS REPETIDAS CON TITULOS
        tabla = tabla[tabla[2] != "Total"]
        tabla = tabla[tabla[2] != "Descripción\nTarifa"]

        #FORMTEAR COLUMNA DESCRIPCION Y TARIFAS
        info = tabla[2].str.split("\n").str[0]
        tarifas = tabla[2].str.split("\n").str[1]
        tabla[2] = info
        tabla.insert(loc=2, column='t', value=tarifas)
    
    if pagina == 2:
        #ELIMINAR FILAS REPETIDAS CON TITULOS
        tabla = tabla[tabla[3] != "Total"]
        tabla = tabla[tabla[2] != "Descripción"]

    #RENOMBRAR NOMBRES COLUMNAS
    tabla.columns = ["category", "info", "tarifa", "actos", "visits", "total"]

    #FORMATEAR TARIFAS 
    tabla["tarifa"] = tabla["tarifa"].fillna("TARIFA PRIVADA")
    tabla["tarifa"] = tabla["tarifa"].str.replace("TARIFA ", "")

    #FORMATEAR ACTOS
    tabla["actos"] = tabla["actos"].fillna(0)

    #FORMATEAR TOTAL
    tabla["total"] = tabla["total"].str.replace("€", "").str.replace(".", "").str.replace(",", ".").str.split(".").str[0].astype(int)

    #RESETEAR INDICES FILAS 
    tabla.reset_index(drop=True, inplace=True)
    
    return tabla

tabla_pag_1_23 = formatear_tabla(dataframes_23[0], 1)
tabla_pag_2_23 = formatear_tabla(dataframes_23[1], 2)
tabla_pag_1_24 = formatear_tabla(dataframes_24[0], 1)
tabla_pag_2_24 = formatear_tabla(dataframes_24[1], 2)

In [92]:
#UNIR TABLAS
def concatenate_dataframes(df1, df2):
  concatenated_df = pd.concat([df1, df2], ignore_index=True)
  return concatenated_df

tabla_23_completa = concatenate_dataframes(tabla_pag_1_23, tabla_pag_2_23)
tabla_24_completa = concatenate_dataframes(tabla_pag_1_24, tabla_pag_2_24)

In [93]:
#FORMAS DE PAGO
def formatear_formas_pago(dataframe):
    formas_pago = dataframe
    formas_pago = formas_pago.iloc[2:6,:]
    formas_pago.columns = ["forma_pago" , "num_operaciones", "total"]
    formas_pago["total"] = formas_pago["total"].str.replace("€", "").str.replace(".", "").str.replace(",", ".").str.split(".").str[0].astype(int)
    formas_pago.loc[2, "forma_pago"] = "Tarjeta"
    formas_pago.loc[3, "forma_pago"] = "Efectivo"
    formas_pago.loc[4, "forma_pago"] = "Transferencia"
    formas_pago.loc[5, "forma_pago"] = "Pronto Pago"
    formas_pago.reset_index(drop=True, inplace=True)
    return formas_pago

tabla_formas_pago_23 = formatear_formas_pago(dataframes_23[3])

C:\Users\34631\AppData\Local\Temp\ipykernel_32908\1103345368.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [94]:
total_saldado_23 = 195317
total_saldado_24 = 151067

In [95]:
tabla_23_completa.loc[[57, 58, 59], 'category'] = 'CORONA'
tabla_23_completa.loc[62, 'category'] = 'ENDODONCIAS'
tabla_23_completa.loc[63, 'category'] = 'EXTRACCION'
tabla_23_completa.loc[64, 'category'] = 'IMPLANTES'
tabla_23_completa.loc[[65, 66, 67, 68], 'category'] = 'PROFILAXIS'
tabla_23_completa.loc[[69, 70], 'category'] = 'OBTURACION'

In [96]:
tabla_23_completa = tabla_23_completa[tabla_23_completa['total'] != 0]
tabla_23_completa = tabla_23_completa[tabla_23_completa['category'] != "(sin especificar)"]
tabla_24_completa = tabla_24_completa[tabla_24_completa['total'] != 0]

In [97]:
tabla_23_completa['actos'] = tabla_23_completa['actos'].astype(int)
tabla_23_completa['visits'] = tabla_23_completa['visits'].astype(int)

tabla_24_completa['actos'] = tabla_24_completa['actos'].astype(int)
tabla_24_completa['visits'] = tabla_24_completa['visits'].astype(int)

In [98]:
tabla_23_completa['precio_uni'] = tabla_23_completa['total'] / tabla_23_completa['actos']
tabla_24_completa['precio_uni'] = tabla_24_completa['total'] / tabla_24_completa['actos']

In [99]:
fig = px.scatter(tabla_23_completa,
                 x="precio_uni", 
                 y="actos",      
                 color="category",
                 symbol ="tarifa" ,
                 title="Relación entre Precio Unitario y Número de Actos 2023", 
                 labels={"precio_uni": "Precio Unitario", "actos": "Actos Realizados"}, 
                 hover_name="info", 
                 hover_data=['category', 'tarifa', 'total', 'visits']) 

fig.update_layout(
    font_family="Oswald", 
    font_color="black",  
    title_font_color="black", 
    plot_bgcolor='white', 
    paper_bgcolor='white'
)

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',     
    zerolinecolor='black', 
    tickfont_color='black',
    title_font_color='black', 
    #tickvals=tickvals_x,
    #ticktext=ticktext_x, 
    #range=[0, 4100],  
    type='log'
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',
    zerolinecolor='black', 
    tickfont_color='black', 
    title_font_color='black', 
    #tickvals=tickvals_y, 
    #ticktext=ticktext_y, 
    #range=[0, 260] 
    type='log'
)

fig.show()

In [100]:
fig = px.scatter(tabla_24_completa,
                 x="precio_uni", 
                 y="actos",      
                 color="category",
                 symbol ="tarifa" ,
                 title="Relación entre Precio Unitario y Número de Actos 2024", 
                 labels={"precio_uni": "Precio Unitario", "actos": "Actos Realizados"}, 
                 hover_name="info", 
                 hover_data=['category', 'tarifa', 'total', 'visits']) 

fig.update_layout(
    font_family="Oswald", 
    font_color="black",  
    title_font_color="black", 
    plot_bgcolor='white', 
    paper_bgcolor='white'
)

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',     
    zerolinecolor='black', 
    tickfont_color='black',
    title_font_color='black', 
    #tickvals=tickvals_x,
    #ticktext=ticktext_x, 
    #range=[0, 4100],  
    type='log'
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',
    zerolinecolor='black', 
    tickfont_color='black', 
    title_font_color='black', 
    #tickvals=tickvals_y, 
    #ticktext=ticktext_y, 
    #range=[0, 260] 
    type='log'
)

fig.show()

In [101]:
fig = px.scatter(tabla_23_completa,
                 x="precio_uni", 
                 y="actos",      
                 color="category",
                 title="Relación entre Precio Unitario y Número de Actos 2023", 
                 labels={"precio_uni": "Precio Unitario", "actos": "Actos Realizados"}, 
                 hover_name="info", 
                 hover_data=['category', 'tarifa', 'total', 'visits']) 

tickvals_x = [0, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 4100]
ticktext_x = [str(val) for val in tickvals_x] 

tickvals_y = [0, 10, 20, 30, 40, 50, 100, 150, 200, 250, 260]
ticktext_y = [str(val) for val in tickvals_y] 


fig.update_layout(
    font_family="Oswald", 
    font_color="black",  
    title_font_color="black", 
    plot_bgcolor='white', 
    paper_bgcolor='white'
)

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',     
    zerolinecolor='black', 
    tickfont_color='black',
    title_font_color='black', 
    tickvals=tickvals_x,
    ticktext=ticktext_x, 
    range=[0, 4100], 
    tickfont=dict(size=6) 
    #type='log'
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',
    zerolinecolor='black', 
    tickfont_color='black', 
    title_font_color='black', 
    tickvals=tickvals_y, 
    ticktext=ticktext_y, 
    range=[0, 260],
    tickfont=dict(size=8)
    #type='log'
)

fig.show()

In [102]:
fig = px.scatter(tabla_24_completa,
                 x="precio_uni", 
                 y="actos",      
                 color="category",
                 title="Relación entre Precio Unitario y Número de Actos 2024", 
                 labels={"precio_uni": "Precio Unitario", "actos": "Actos Realizados"}, 
                 hover_name="info", 
                 hover_data=['category', 'tarifa', 'total', 'visits']) 

tickvals_x = [0, 100, 200, 300, 400, 500, 1000, 2000, 3000, 4000, 5000]
ticktext_x = [str(val) for val in tickvals_x] 

tickvals_y = [0, 10, 20, 30, 40, 50, 100, 150, 200, 250, 350]
ticktext_y = [str(val) for val in tickvals_y] 


fig.update_layout(
    font_family="Oswald", 
    font_color="black",  
    title_font_color="black", 
    plot_bgcolor='white', 
    paper_bgcolor='white'
)

fig.update_xaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',     
    zerolinecolor='black', 
    tickfont_color='black',
    title_font_color='black', 
    tickvals=tickvals_x,
    ticktext=ticktext_x, 
    range=[0, 5000], 
    tickfont=dict(size=6) 
    #type='log'
)

fig.update_yaxes(
    showgrid=True,
    gridcolor='lightgray', 
    linecolor='black',
    zerolinecolor='black', 
    tickfont_color='black', 
    title_font_color='black', 
    tickvals=tickvals_y, 
    ticktext=ticktext_y, 
    range=[0, 350],
    tickfont=dict(size=8)
    #type='log'
)

fig.show()

In [104]:
color_map = {
    'ORTODONCIAS': '#00BFFF',         # Bright Coral (Energetic)
    'CORONA': '#FFD93D',              # Cyber Yellow (Striking & Positive)
    'IMPLANTES':'#FF7F50' ,           # Teal Blue (Deep & Modern)
    'OBTURACION': '#FF9F43',          # Tangerine Orange (Vivid & Playful)
    'PROFILAXIS': '#6BB97A',          # Emerald Green (Fresh & Clean)
    'PROTESIS REMOVIBLE': '#FF6F59',  # Tomato Red (Bold & Direct)
    'EXTRACCION': '#FF4D4D',          # Pure Red (Assertive & Clear)
    'ENDODONCIAS': '#FFCD00',         # Golden Yellow (Bright & Important)
    'DIAGNOSTICOS': '#FF6B6B',        # Deep Sky Blue (Clean & Clear)
    'CIRUGIA MAXILOFACIAL': '#FF8C00',# Dark Orange (Strong & Distinct)
    'BLANQUEAMIENTO': '#87CEEB',      # Sky Blue (Light & Airy)
    'PUENTES': '#8A2BE2',             # Blue Violet (Unique & Stable)
    'CIRUGIA PERIODONTAL': "#06DD51", # Chartreuse Green (Zesty & Lively)
    'FRENECTOMIA': '#40E0D0',         # Turquoise (Cool & Calm)
    'MUÑON': '#3B8D99'              # Coral (Warm & Approachable)
}

In [105]:
category_revenue = tabla_23_completa.groupby('category')['total'].sum().reset_index()

category_revenue = category_revenue.sort_values(by='total', ascending=False)

fig = px.bar(category_revenue ,
             x='category',
             y='total',
             color='category',
             color_discrete_map=color_map,
             title='Total Ganancias por Categoria Medica 2023',
             labels={'category': 'Categoría', 'total': 'Total (€)'})

fig.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode="x unified"
)

fig.show()

In [107]:
category_revenue = tabla_23_completa.groupby('category')['actos'].sum().reset_index()

category_revenue = category_revenue.sort_values(by='actos', ascending=False)

fig = px.bar(category_revenue ,
             x='category',
             y='actos',
             color='category',
             color_discrete_map=color_map,
             title='Total Actos por Categoria Medica 2023',
             labels={'category': 'Categoría', 'actos': 'Nº Actos'})

fig.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode="x unified"
)

fig.show()

In [ ]:
category_revenue = tabla_24_completa.groupby('category')['total'].sum().reset_index()

category_revenue = category_revenue.sort_values(by='total', ascending=False)

fig = px.bar(category_revenue ,
             x='category',
             y='total',
             color='category',
             color_discrete_map=color_map,
             title='Total Ganancias por Categoria Medica 2024',
             labels={'category': 'Categoría', 'total': 'Total (€)'})

fig.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode="x unified"
)

fig.show()



In [109]:
category_revenue = tabla_24_completa.groupby('category')['actos'].sum().reset_index()

category_revenue = category_revenue.sort_values(by='actos', ascending=False)

fig = px.bar(category_revenue ,
             x='category',
             y='actos',
             color='category',
             color_discrete_map=color_map,
             title='Total Actos por Categoria Medica 2024',
             labels={'category': 'Categoría', 'actos': 'Nº Actos'})

fig.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    xaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    yaxis=dict(
        showline=True,
        linewidth=1,
        linecolor='black',
        tickfont=dict(color='black')
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    hovermode="x unified"
)

fig.show()

In [ ]:
category_info_data = tabla_24_completa.groupby(['category', 'info']).agg({'total': 'sum', 'actos': 'sum'}).reset_index()
fig_acts_treemap = px.treemap(category_info_data,
                              path=[px.Constant("Categorías"), 'category', 'info'],
                              values='actos',
                              color='category', 
                              color_discrete_map=color_map, 
                              title='Número de Actos por Categoría y Tratamiento 2024', 
                              labels={'parent': 'Categoría', 'labels': 'Tratamiento', 'value': 'Número de Actos'}) 

fig_acts_treemap.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    plot_bgcolor='black', 
    paper_bgcolor='white',
    margin=dict(t=40, l=20, r=20, b=20) 
)

fig_acts_treemap.update_traces(marker_line_color='black', marker_line_width=1, textfont_color='black')

if fig_acts_treemap.data and len(fig_acts_treemap.data[0].marker.colors) > 0:
    current_colors = list(fig_acts_treemap.data[0].marker.colors)
    current_colors[0] = 'white'
    fig_acts_treemap.data[0].marker.colors = current_colors

#fig_acts_treemap.show()

fig_revenue_treemap = px.treemap(category_info_data,
                                 path=[px.Constant("Categorias"), 'category', 'info'], 
                                 values='total',
                                 color='category',
                                 color_discrete_map=color_map, 
                                 title='Ganancia Total por Categoría y Tratamiento (2024)', 
                                 labels={'parent': 'Categoría', 'labels': 'Tratamiento', 'value': 'Ganancia Total (€)'}) 

fig_revenue_treemap.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, l=20, r=20, b=20)  
)

fig_revenue_treemap.update_traces(marker_line_color='black', marker_line_width=1, textfont_color='black')

if fig_revenue_treemap.data and len(fig_revenue_treemap.data[0].marker.colors) > 0:
    current_colors = list(fig_revenue_treemap.data[0].marker.colors)
    current_colors[0] = 'white'
    fig_revenue_treemap.data[0].marker.colors = current_colors
    
#fig_revenue_treemap.show()

fig_subplots = make_subplots(rows=1, cols=2,
                             subplot_titles=("Número de Actos por Categoría y Tratamiento 2024",
                                             "Ganancia Total por Categoría y Tratamiento 2024"),
                             specs=[[{'type': 'domain'}, {'type': 'domain'}]])

fig_subplots.add_trace(fig_acts_treemap.data[0], row=1, col=1)
fig_subplots.add_trace(fig_revenue_treemap.data[0], row=1, col=2)

fig_subplots.update_layout(
    title_text="Análisis de Actos y Ganancias por Categoría y Tratamiento en 2024",
    title_x=0.5,
    font_family="Oswald",
    title_font_family="Oswald",
    height=800
)

fig_subplots.show()


In [134]:
category_info_data = tabla_23_completa.groupby(['category', 'info']).agg({'total': 'sum', 'actos': 'sum'}).reset_index()
fig_acts_treemap = px.treemap(category_info_data,
                              path=[px.Constant("Categorías"), 'category', 'info'],
                              values='actos',
                              color='category', 
                              color_discrete_map=color_map, 
                              title='Número de Actos por Categoría y Tratamiento 2023', 
                              labels={'parent': 'Categoría', 'labels': 'Tratamiento', 'value': 'Número de Actos'}) 

fig_acts_treemap.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    plot_bgcolor='black', 
    paper_bgcolor='white',
    margin=dict(t=40, l=20, r=20, b=20) 
)

fig_acts_treemap.update_traces(marker_line_color='black', marker_line_width=1, textfont_color='black')

if fig_acts_treemap.data and len(fig_acts_treemap.data[0].marker.colors) > 0:
    current_colors = list(fig_acts_treemap.data[0].marker.colors)
    current_colors[0] = 'white'
    fig_acts_treemap.data[0].marker.colors = current_colors

fig_revenue_treemap = px.treemap(category_info_data,
                                 path=[px.Constant("Categorias"), 'category', 'info'], 
                                 values='total',
                                 color='category',
                                 color_discrete_map=color_map, 
                                 title='Ganancia Total por Categoría y Tratamiento 2023', 
                                 labels={'parent': 'Categoría', 'labels': 'Tratamiento', 'value': 'Ganancia Total (€)'}) 

fig_revenue_treemap.update_layout(
    font_family="Oswald",
    title_font_family="Oswald",
    plot_bgcolor='white',
    paper_bgcolor='white',
    margin=dict(t=40, l=20, r=20, b=20)  
)

fig_revenue_treemap.update_traces(marker_line_color='black', marker_line_width=1, textfont_color='black')

if fig_revenue_treemap.data and len(fig_revenue_treemap.data[0].marker.colors) > 0:
    current_colors = list(fig_revenue_treemap.data[0].marker.colors)
    current_colors[0] = 'white'
    fig_revenue_treemap.data[0].marker.colors = current_colors

fig_subplots = make_subplots(rows=1, cols=2,
                             subplot_titles=("Número de Actos por Categoría y Tratamiento 2023",
                                             "Ganancia Total por Categoría y Tratamiento 2023"),
                             specs=[[{'type': 'domain'}, {'type': 'domain'}]])

fig_subplots.add_trace(fig_acts_treemap.data[0], row=1, col=1)
fig_subplots.add_trace(fig_revenue_treemap.data[0], row=1, col=2)

fig_subplots.update_layout(
    title_text="Análisis de Actos y Ganancias por Categoría y Tratamiento en 2023",
    title_x=0.5,
    font_family="Oswald",
    title_font_family="Oswald",
    width= 1000,
    height=600
)

fig_subplots.show()

In [ ]:
def categorize_aesthetic(description):
    description = str(description).upper()

    aesthetic_treatments = [
        "BLANQUEAMIENTO PHILIPS ZOOM",
        "BRACKETS ESTETICOS",
        "INVISALIGN",
        "RECONSTRUCCION COMPOSITE",
        "CORONA METAL CERAMICA",
        "CORONA RESINA(LABORATORIO)",
        "CORONA IMPLANTE",
        "CORONA IMPLANTE ZIRCONIO",
        "CORONA TOTAL METAL NOBLE",
        "BRACKETS METALICOS"
    ]

    if description in aesthetic_treatments:
        return True
        
    return False

tabla_23_completa['estetico'] = tabla_23_completa['info'].apply(categorize_aesthetic)
tabla_24_completa['estetico'] = tabla_24_completa['info'].apply(categorize_aesthetic)

In [136]:

earnings_by_estetico = tabla_23_completa.groupby('estetico')['total'].sum().reset_index()

earnings_by_estetico['estetico_label'] = earnings_by_estetico['estetico'].apply(lambda x: 'Estético' if x else 'No Estético')

# Create the pie chart
# Crear el gráfico de pastel
fig = px.pie(earnings_by_estetico,
             values='total',
             names='estetico_label',
             title='Porcentaje de Ganancias Totales por Tipo de Tratamiento (Estético vs. No Estético)',
             hole=0.3) # Added a hole to make it a donut chart (Se añadió un agujero para hacerlo un gráfico de anillos)

# Update trace properties for text position and info
# Actualizar las propiedades de la traza para la posición del texto y la información
fig.update_traces(textposition='inside', textinfo='percent+label')

# Display the figure
# Mostrar la figura
fig.show()

In [142]:
actos_by_estetico = tabla_23_completa.groupby('estetico')['actos'].sum().reset_index()

actos_by_estetico['estetico_label'] = actos_by_estetico['estetico'].apply(lambda x: 'Estético' if x else 'No Estético')
# Create the pie chart
# Crear el gráfico de pastel
fig = px.pie(actos_by_estetico,
             values='actos',
             names='estetico_label',
             title='Porcentaje de Ganancias Totales por Tipo de Tratamiento (Estético vs. No Estético)',
             hole=0.3) # Added a hole to make it a donut chart (Se añadió un agujero para hacerlo un gráfico de anillos)

# Update trace properties for text position and info
# Actualizar las propiedades de la traza para la posición del texto y la información
fig.update_traces(textposition='inside', textinfo='percent+label')

# Display the figure
# Mostrar la figura
fig.show()

In [144]:
ganancias_estetico = tabla_23_completa.groupby('estetico')['total'].sum().reset_index()

ganancias_estetico['text_estetico'] = ganancias_estetico['estetico'].apply(lambda x: 'Estético' if x else 'No Estético')

actos_estetico = tabla_23_completa.groupby('estetico')['actos'].sum().reset_index()

actos_estetico['text_estetico'] = actos_estetico['estetico'].apply(lambda x: 'Estético' if x else 'No Estético')

fig = make_subplots(rows=1, cols=2, specs=[[{'type':'domain'}, {'type':'domain'}]],
                    subplot_titles=['Ganancias Totales', 'Número de Actos Realizados'])

fig.add_trace(go.Pie(labels=ganancias_estetico['text_estetico'],
                     values=ganancias_estetico['total'],
                     name='Ganancias',
                     hole=0.3,
                     textposition='inside',
                     textinfo='percent+label',
                     textfont_color='black'),
              1, 1)

fig.add_trace(go.Pie(labels=actos_estetico['text_estetico'],
                     values=actos_estetico['actos'],
                     name='Actos',
                     hole=0.3,
                     textposition='inside',
                     textinfo='percent+label',
                     textfont_color='black'),
              1, 2)

fig.update_layout(title_text='Ganancias y Actos por Tipo de Tratamiento 2023',font=dict(family="Oswald", color="black"))

fig.show()

In [146]:

df_estetico = tabla_23_completa[tabla_23_completa['estetico'] == True]

total_ganancia_estetico = df_estetico['total'].sum()

ganancia_por_tratamiento_estetico = df_estetico.groupby('info')['total'].sum().reset_index()

ganancia_por_tratamiento_estetico['porcentaje'] = (ganancia_por_tratamiento_estetico['total'] / total_ganancia_estetico) * 100

fig = go.Figure(data=[go.Pie(labels=ganancia_por_tratamiento_estetico['info'],
                             values=ganancia_por_tratamiento_estetico['porcentaje'],
                             hoverinfo='label+percent',
                             textinfo='percent',
                             insidetextorientation='radial'
                            )])

fig.update_layout(title_text='Porcentaje de Ganancia por Tratamiento Estético')

fig.show()

In [150]:
color_map_estetico = {
    'RECONSTRUCCION COMPOSITE': '#00BFFF',         # ORTODONCIAS
    'CORONA METAL CERAMICA': '#FFD93D',            # CORONA
    'CORONA RESINA(LABORATORIO)': '#FF7F50',       # IMPLANTES
    'CORONA IMPLANTE': '#FF9F43',                  # OBTURACION
    'CORONA IMPLANTE ZIRCONIO': '#6BB97A',         # PROFILAXIS
    'BRACKETS METALICOS': '#FF6F59',               # PROTESIS REMOVIBLE
    'BRACKETS ESTETICOS': '#FF4D4D',               # EXTRACCION
    'INVISALIGN': '#FFCD00',                       # ENDODONCIAS
    'CORONA TOTAL METAL NOBLE': '#FF6B6B',         # DIAGNOSTICOS
    'BLANQUEAMIENTO PHILIPS ZOOM': '#FF8C00',      # CIRUGIA MAXILOFACIAL
    'CORONA IMPLANTE': '#87CEEB',                  # BLANQUEAMIENTO
    'RECONSTRUCCION COMPOSITE': '#8A2BE2'          # PUENTES
}

In [152]:
df_estetico = tabla_23_completa[tabla_23_completa['estetico'] == True]

total_ganancia_estetico = df_estetico['total'].sum()

ganancia_por_tratamiento_estetico = df_estetico.groupby('info')['total'].sum().reset_index()

ganancia_por_tratamiento_estetico['porcentaje'] = (ganancia_por_tratamiento_estetico['total'] / total_ganancia_estetico) * 100

# Asignar color a cada tratamiento
ganancia_por_tratamiento_estetico['color'] = ganancia_por_tratamiento_estetico['info'].map(color_map_estetico)

fig = go.Figure(data=[go.Pie(
    labels=ganancia_por_tratamiento_estetico['info'],
    values=ganancia_por_tratamiento_estetico['porcentaje'],
    hoverinfo='label+percent',
    textinfo='percent',
    insidetextorientation='radial',
    marker=dict(colors=ganancia_por_tratamiento_estetico['color'])
)])

fig.update_layout(
    title_text='Porcentaje de Ganancia por Tratamiento Estético',
    font=dict(family='Oswald, sans-serif', color='black')
)

fig.show()



In [149]:
tabla_24_completa.loc[tabla_24_completa['estetico']==True, "info"]

16          CORONA METAL CERAMICA
17     CORONA RESINA(LABORATORIO)
18                CORONA IMPLANTE
19       CORONA IMPLANTE ZIRCONIO
35             BRACKETS METALICOS
36             BRACKETS ESTETICOS
37                     INVISALIGN
44    BLANQUEAMIENTO PHILIPS ZOOM
Name: info, dtype: object